## 集成方法二

本部分主要是投票器，策略学习等

### 投票分类器

VotingClassifier是组合概念上不同的机器学习分类器，并使用多数表决或平均预测概率（软表决）来预测类标签。这样的分类器可用于一组同样表现良好的模型，以平衡其各自的弱点。

#### Majority Class Labels (Majority/Hard Voting)
####  Weighted Average Probabilities (Soft Voting)

目前在skleran中两个方法为同一个类，只是参数不同。

**参数：**
 - estimators:list of (str, estimator) tuples；fit在VotingClassifier将要存储在class属性中的那些原始估计量的克隆中 调用该方法self.estimators_。可以将估算器设置为'drop' 使用set_params;从0.22版开始None弃用：在0.22 版中弃用了使用估算器，而在0.24 版中弃用了支持。使用字符串'drop'代替。
 - voting:{‘hard’, ‘soft’}, 默认为’hard’:如果为'hard'，则将预测的类别标签用于多数规则投票。否则，如果为“ soft”，则基于预测概率之和的argmax来预测类别标签，建议将其用于一组经过良好校准的分类器。
 - weight:sarray-like of shape (n_classifiers,), 默认为None;加权序列（float或int），用于hard在平均（soft投票）之前对预测的类别标签（投票）或类别概率的出现进行加权。如果使用统一权重None。
 - n_jobs:int, 默认为None;为并行运行的作业数fit。 None除非在joblib.parallel_backend上下文中，否则表示1 。 -1表示使用所有处理器。
 - flatten_transformbool, 默认为True;仅当voting ='soft'时，才会影响变换输出的形状。如果voting ='soft'并且flatten_transform = True，则transform方法将返回具有形状的矩阵（n_samples，n_classifiers * n_classes）。如果flatten_transform = False，则返回（n_classifiers，n_samples，n_classes）。
 - verbose:bool, 默认为False;如果为True，则在完成拟合时将打印经过的时间。

In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
clf1 = LogisticRegression(multi_class='multinomial', random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
y = np.array([1, 1, 1, 2, 2, 2])
eclf1 = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1 = eclf1.fit(X, y)
print(eclf1.predict(X))

[1 1 1 2 2 2]


In [2]:
eclf1 = VotingClassifier(estimators=[
        ('lr', RandomForestClassifier(n_estimators=50, random_state=1)), ('rf', LogisticRegression(multi_class='multinomial', random_state=1)), ('gnb', GaussianNB())], voting='hard')
eclf1 = eclf1.fit(X, y)
print(eclf1.predict(X))

[1 1 1 2 2 2]


In [3]:
np.array_equal(eclf1.named_estimators_.lr.predict(X),
               eclf1.named_estimators_['lr'].predict(X))

True

In [4]:
eclf2 = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3)],
        voting='soft')              {
                "hyperparameter": "init",
                "description": "estimator or ‘zero’, 默认为None:一个估计器对象，用于计算初始预测。 init必须提供fit和predict_proba。如果为“零”，则初始原始预测设置为零。默认情况下，使用 DummyEstimator预测类优先级。",
                "class_name": [
                  "custom_object",
                  "None",
                  "single_choice"
                ],
                "values": [
                  "zero"
                ],
                "default": "None",
                "conditions": []
              },
eclf2 = eclf2.fit(X, y)
print(eclf2.predict(X))

[1 1 1 2 2 2]


In [5]:
eclf3 = VotingClassifier(estimators=[
       ('lr', clf1), ('rf', clf2), ('gnb', clf3)],
       voting='soft', weights=[2,1,1],
       flatten_transform=True)
eclf3 = eclf3.fit(X, y)
print(eclf3.predict(X))

[1 1 1 2 2 2]


In [6]:
print(eclf3.transform(X).shape)

(6, 6)


### VotingRegressor

**参数:**
 - estimators:list of (str, estimator) tuples；fit在VotingClassifier将要存储在class属性中的那些原始估计量的克隆中 调用该方法self.estimators_。可以将估算器设置为'drop' 使用set_params;从0.22版开始None弃用：在0.22 版中弃用了使用估算器，而在0.24 版中弃用了支持。使用字符串'drop'代替。
 - weight:sarray-like of shape (n_classifiers,), 默认为None;加权序列（float或int），用于hard在平均（soft投票）之前对预测的类别标签（投票）或类别概率的出现进行加权。如果使用统一权重None。
 - n_jobs:int, 默认为None;为并行运行的作业数fit。 None除非在joblib.parallel_backend上下文中，否则表示1 。-1表示使用所有处理器,但是会对可视化编程的监控器造成影响。
 - verbose:bool, 默认为False;如果为True，则在完成拟合时将打印经过的时间。

In [7]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
r1 = LinearRegression()
r2 = RandomForestRegressor(n_estimators=10, random_state=1)
X = np.array([[1, 1], [2, 4], [3, 9], [4, 16], [5, 25], [6, 36]])
y = np.array([2, 6, 12, 20, 30, 42])
er = VotingRegressor([('lr', r1), ('rf', r2)])
print(er.fit(X, y).predict(X))

[ 3.3  5.7 11.8 19.7 28.  40.3]


### 堆叠泛化(Stacked generalization)

堆叠泛化是一种组合估算器以减少其偏差的方法。更精确地的来说，每个单独的估计器的预测被堆叠在一起，并用作最终估计器的输入以计算预测。该最终估计量通过交叉验证进行训练。

**参数:**

 - estimators:list of (str, estimator) tuples；fit在VotingClassifier将要存储在class属性中的那些原始估计量的克隆中 调用该方法self.estimators_。可以将估算器设置为'drop' 使用set_params;从0.22版开始None弃用：在0.22 版中弃用了使用估算器，而在0.24 版中弃用了支持。使用字符串'drop'代替。
 
 - final_estimator 估算器，默认=无;一个分类器，将用于合并基本估计量。默认分类器是LogisticRegression。
 
 - cv:int，交叉验证生成器或可迭代的默认值=无;确定用于cross_val_predict训练的交叉验证拆分策略 final_estimator。参数的可能输入是：无，将使用默认的5倍交叉验证;整数，用于指定（分层）KFold中的折数，用作交叉验证生成器的对象;An iterable yielding train, test splits。对于整数/无输入，如果估计量是分类器，并且y是二进制或多类，则使用StratifiedKFold。在所有其他情况下，KFold均使用。
 
 - stack_method:{‘auto’, ‘predict_proba’, ‘decision_function’, ‘predict’}, 默认为’auto’;如果“自动”，它会尝试调用，每个估计，顺序为:'predict_proba'，'decision_function'或'predict'。否则则根据输入调用:'predict_proba'，'decision_function'或 'predict'。不过如果估计器未实现该方法，则将引发错误。
 - n_jobs:int, 默认为None;为并行运行的作业数fit。 None除非在joblib.parallel_backend上下文中，否则表示1 。-1表示使用所有处理器,但是会对可视化编程的监控器造成影响。
 - passthrough:bool默认为None;如果为False，则仅将估算值的预测用作final_estimator的训练数据。设为True时，将final_estimator根据预测以及原始训练数据对进行训练。
 - verboseint, 默认为0;Verbosity level.

In [8]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
X, y = load_diabetes(return_X_y=True)
estimators = [
    ('lr', RidgeCV()),
    ('svr', LinearSVR(random_state=42))
]
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,
                                          random_state=42)
)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)
reg.fit(X_train, y_train).score(X_test, y_test)

0.3696684361321816